In [1]:
!unzip output.zip -d /content

Archive:  output.zip
   creating: /content/output/
   creating: /content/output/output/
  inflating: /content/output/output/Art5_Oliwia_Wierzbaa.xlsx  
  inflating: /content/output/output/Art5_Paulina_Piotrowska.xlsx  
  inflating: /content/output/output/Art6_Oliwia_Wierzbaa.xlsx  
  inflating: /content/output/output/Art6_Paulina_Piotrowska.xlsx  
  inflating: /content/output/output/Bus11_Leonie_Wessa.xlsx  
  inflating: /content/output/output/Bus11_Oliwia_Wierzba.xlsx  
  inflating: /content/output/output/Bus14_Leonie_Wessa.xlsx  
  inflating: /content/output/output/Bus14_Oliwia_Wierzba.xlsx  
  inflating: /content/output/output/Edu2_Elina_Arendt.xlsx  
  inflating: /content/output/output/Edu2_Macie_Kilandi.xlsx  
  inflating: /content/output/output/Edu6_Elina_Arendt.xlsx  
  inflating: /content/output/output/Edu6_Macie_Kilandi.xlsx  
  inflating: /content/output/output/Ent5_Laura_Bach.xlsx  
  inflating: /content/output/output/Ent5_Sofie_Wüller.xlsx  
  inflating: /content/output/out

In [2]:
!pip install krippendorff

In [3]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import krippendorff
import numpy as np
import glob
import os

In [4]:
def extract_disc_author(filename):
    """
    From: 'Edu_Macie_Kilandi_merged.xlsx'
    → discipline='Edu'
    → author='Macie_Kilandi'
    (removes 'merged' and file extension)
    """
    base = os.path.splitext(filename)[0]         # remove .xlsx
    base = base.replace("_merged", "")           # remove merged
    parts = base.split("_")

    discipline = parts[0]                        # first token
    author = "_".join(parts[1:])                 # everything after

    return discipline, author


In [5]:

def calculate_krippendorffs_alpha(df_list, columns):
    """
    Krippendorff's alpha:
    - Evaluative: computed on all rows
    - Other columns: computed ONLY where Evaluative == 1 for ALL annotators
    """
    alphas = {}

    for col in columns:
        data = []

        # mask logic
        if col == "Evaluative":
            mask = np.ones(len(df_list[0]), dtype=bool)
        else:
            # ONLY rows where all annotators marked Evaluative == 1
            mask = np.logical_and.reduce(
                [df["Evaluative"] == 1 for df in df_list]
            )

        for df in df_list:
            col_data = df.loc[mask, col].replace(-1, np.nan).to_numpy()
            data.append(col_data)

        data = np.array(data)

        # skip if no usable data
        if data.shape[1] == 0:
            alphas[col] = np.nan
            continue

        # skip if only one unique value
        unique_vals = np.unique(data[~np.isnan(data)])
        if len(unique_vals) <= 1:
            alphas[col] = np.nan
            continue

        alphas[col] = krippendorff.alpha(
            reliability_data=data,
            level_of_measurement="nominal"
        )

    return alphas


In [6]:
# FOR EACH FILE
results = []

common_dfs = []

gold_file = "/content/Gold_standard_Mirela.xlsx"
df_gold = pd.read_excel(gold_file)

# Normalize gold file
df_gold = df_gold.fillna(0)


df_gold = df_gold.fillna(0)
#df_gold2 = df_gold2.drop(index=5)

#Just to be sure of int values
df_gold['Evaluative'] = df_gold['Evaluative'].astype(int)
df_gold['Affect'] = df_gold['Affect'].astype(int)
df_gold['Judgement'] = df_gold['Judgement'].astype(int)
df_gold['Appreciation'] = df_gold['Appreciation'].astype(int)
df_gold['Ambiguous'] = df_gold['Ambiguous'].astype(int)
#df_gold['Uncertain'] = df_gold['Uncertain'].astype(int)
for col in ['Evaluative','Affect','Judgement','Appreciation','Ambiguous']:
    df_gold[col] = df_gold[col].astype(int)

# Path where student files live
student_files = glob.glob("/content/output/output/*.xlsx")
print(student_files)


for file in student_files:
    df2 = pd.read_excel(file)

    # Clean student file
    # Identify any column whose name equals 'uncertain' ignoring case sometimes it was 'Uncertain'
    uncertain_cols = [c for c in df2.columns if c.lower() == 'uncertain']

    # Remove rows where uncertain == 1 cause that result is not useful for us!!
    for col in uncertain_cols:
        df2 = df2[df2[col] != 1]

    df2 = df2.drop(['uncertain','Notes'], axis=1, errors='ignore').fillna(0)
    for col in ['Evaluative','Affect','Judgement','Appreciation','Ambiguous']:
        df2[col] = df2[col].astype(int)

    df2 = df2.fillna(0)

    df2['Evaluative'] = df2['Evaluative'].astype(int)
    df2['Affect'] = df2['Affect'].astype(int)
    df2['Judgement'] = df2['Judgement'].astype(int)
    df2['Appreciation'] = df2['Appreciation'].astype(int)
    df2['Ambiguous'] = df2['Ambiguous'].astype(int)
    #df2['Uncertain'] = df2['Uncertain'].astype(int)

    # Align based on Text_ID + Sentence_ID
    common_pairs = df_gold.merge(
        df2[['Text_ID','Sentence_ID']],
        on=['Text_ID','Sentence_ID'],
        how='inner'
    )[['Text_ID','Sentence_ID']]




    df1_aligned = df_gold.merge(common_pairs, on=['Text_ID','Sentence_ID'], how='inner')
    df2_aligned = df2.merge(common_pairs, on=['Text_ID','Sentence_ID'], how='inner')

    # SORTING IS VERY IMPORTANT OTHERWISE IT JUST CHECKS ON WHATEVER ORDER IT OCCURS
    df1_aligned = df1_aligned.sort_values(['Text_ID','Sentence_ID']).reset_index(drop=True)
    df2_aligned = df2_aligned.sort_values(['Text_ID','Sentence_ID']).reset_index(drop=True)

    # --- Compute Cohen's Kappa ---
    kappa = cohen_kappa_score(df1_aligned["Evaluative"], df2_aligned["Evaluative"])

    # --- Compute Krippendorff's Alpha ---
    cols_to_check = ['Evaluative','Affect','Judgement','Appreciation','Ambiguous']
    alphas = calculate_krippendorffs_alpha([df1_aligned, df2_aligned], cols_to_check)
    filename = os.path.basename(file)
    discipline, author = extract_disc_author(filename)

    results.append({
        "discipline": discipline,
        "author": author,
        "cohen_kappa": kappa,
        **alphas
    })


    # ---------------------------------------------
    # Build common gold–student dataframe
    # ---------------------------------------------
    common_df = df1_aligned.merge(
    df2_aligned,
    on=['Text_ID', 'Sentence_ID'],
    how='inner',
    suffixes=('_gold', '_student')
    )

    # Add metadata columns
    common_df.insert(0, 'author', author)
    common_df.insert(0, 'discipline', discipline)

    common_dfs.append(common_df)


# Convert to DataFrame
results_df = pd.DataFrame(results)

results_df = pd.DataFrame(results)
results_df.to_excel("annotation_agreement_results.xlsx", index=False)
print(results_df)

final_common_df = pd.concat(common_dfs, ignore_index=True)

assert not final_common_df.duplicated(
    ['author', 'Text_ID', 'Sentence_ID']
).any()

final_common_df.to_excel(
    "common_gold_student_annotations.xlsx",
    index=False
)




['/content/output/output/Edu2_Elina_Arendt.xlsx', '/content/output/output/Art6_Paulina_Piotrowska.xlsx', '/content/output/output/His13_Marta_MartinezSerrano.xlsx', '/content/output/output/Tech10_Josefine_Baer.xlsx', '/content/output/output/Pol9_Ela_Erkasap.xlsx', '/content/output/output/Ent5_Sofie_Wüller.xlsx', '/content/output/output/Art6_Oliwia_Wierzbaa.xlsx', '/content/output/output/His5_Yolanda_Ortega.xlsx', '/content/output/output/Phi8_Sema_Erkus.xlsx', '/content/output/output/Tech10_Sean-Pascal_Kuttner.xlsx', '/content/output/output/Pol2_Chrysoula_Choutouris.xlsx', '/content/output/output/Med9_Maria_SuarezPena.xlsx', '/content/output/output/Tech21_Sean-Pascal_Kuttner.xlsx', '/content/output/output/Bus14_Leonie_Wessa.xlsx', '/content/output/output/Edu2_Macie_Kilandi.xlsx', '/content/output/output/Pol9_Chrysoula_Choutouris.xlsx', '/content/output/output/His13_Yolanda_Ortega.xlsx', '/content/output/output/Psy5_Fabian_Heinze.xlsx', '/content/output/output/Sci7_Chiara_Büschel.xlsx', '

In [7]:
# Author level

LABEL_COLS = [
    'Evaluative',
    'Affect',
    'Judgement',
    'Appreciation',
    'Ambiguous'
]
gold_label_cols = [c for c in final_common_df.columns if c.endswith('_gold')]
student_label_cols = [c for c in final_common_df.columns if c.endswith('_student')]
author_results = []

for author, df_author in final_common_df.groupby('author'):

    # ---------------------------------------------
    # Cohen’s Kappa (Evaluative)
    # ---------------------------------------------

    kappa = cohen_kappa_score(
        df_author['Evaluative_gold'],
        df_author['Evaluative_student']
    )

    # ---------------------------------------------
    # Krippendorff’s Alpha
    # ---------------------------------------------

    df_gold_author = df_author[[c for c in gold_label_cols]].copy()
    df_student_author = df_author[[c for c in student_label_cols]].copy()

    # Remove suffixes so column names match
    df_gold_author.columns = [c.replace('_gold', '') for c in df_gold_author.columns]
    df_student_author.columns = [c.replace('_student', '') for c in df_student_author.columns]

    alphas = calculate_krippendorffs_alpha(
        [df_gold_author, df_student_author],
        LABEL_COLS
    )

    author_results.append({
        'author': author,
        'disciplines_merged': sorted(df_author['discipline'].unique().tolist()),
        'n_sentences': len(df_author),
        'cohen_kappa': kappa,
        **alphas
    })


In [8]:
author_results_df = pd.DataFrame(author_results)
author_results_df.to_excel(
    "annotation_agreement_results_by_author.xlsx",
    index=False
)

print(author_results_df)


                   author disciplines_merged  n_sentences  cohen_kappa  \
0              Artem_Kilp       [Psy5, Psy8]           40     0.103870   
1          Chiara_Büschel      [Sci10, Sci7]           40     0.376623   
2    Chrysoula_Choutouris       [Pol2, Pol9]           40     0.308511   
3             Ela_Erkasap       [Pol2, Pol9]           40     0.283887   
4            Elina_Arendt       [Edu2, Edu6]           35     0.470842   
5           Fabian_Heinze       [Psy5, Psy8]           40     0.093851   
6           Josefine_Baer   [Tech10, Tech21]           40     0.250000   
7              Laura_Bach       [Ent5, Ent8]           40     0.166667   
8            Leonie_Wessa     [Bus11, Bus14]           37     0.336918   
9           Macie_Kilandi       [Edu2, Edu6]           39     0.561798   
10       Maria_SuarezPena      [Med15, Med9]           40     0.452736   
11  Marta_MartinezSerrano      [His13, His5]           40     0.319372   
12          Marta_Tarraga      [Med15,